In [21]:
import sys
sys.path.insert(1, '../')
from utils.packages import *
from bertviz.neuron_view import show
from bertviz import head_view, model_view
from transformers import BertTokenizer, BertModel

In [24]:
path='../data/datasets/synthea/'
train = pd.read_parquet(path + 'train')
val = pd.read_parquet(path + 'val')
test = pd.read_parquet(path + 'test')

In [97]:
files = {'code':'../data/vocabularies/Synthea/snomedrxnorm.npy',
         'age':'../data/vocabularies/Synthea/age.npy'
        }
tokenizer = EHRTokenizer(task='MLM', filenames=files)
model_config = {
        'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
        'hidden_size': 288, #tune.choice([100, 150, 288]), #288, # word embedding and seg embedding hidden size
        'seg_vocab_size': 2, # number of vocab for seg embedding
        'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
        'gender_vocab_size': 3,
        'max_position_embeddings': 32, # maximum number of tokens
        'hidden_dropout_prob': 0.1, # dropout rate
        'num_hidden_layers': 4, #4, # number of multi-head attention layers required
        'num_attention_heads': 6, # number of attention heads
        'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
        'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
        'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
        'initializer_range': 0.02, # parameter weight initializer range
        'use_prior':False,
        'reg':0.1,
        'epochs':10,
    }
optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

In [105]:
condfiles = {'dd':'../data/train_stats/dd_cond_probs.empirical.p', 
                 'dp':'../data/train_stats/dp_cond_probs.empirical.p', 
                 'pp':'../data/train_stats/pp_cond_probs.empirical.p' , 
                 'pd':'../data/train_stats/pd_cond_probs.empirical.p'
                }
testd = EHRDataset(train, max_len=32, tokenizer=tokenizer, conditional_files=condfiles, run_type='train')

Loading data


In [106]:
testloader = torch.utils.data.DataLoader(testd, batch_size=1, shuffle=True, pin_memory=True, num_workers=4)

In [107]:
conf = BertConfig(model_config)
model = BertForMaskedLM(conf) 
PATH = "../saved_models/MLM/model_without_prior"
model = load_model(PATH, model)
params = list(model.named_parameters())
optim = adam(params, optim_param)

t_total value of -1 results in schedule not being applied


In [111]:
example = next(iter(testloader))

In [112]:
age_ids, gender_ids, input_ids, posi_ids, segment_ids, attmask, labels, prior_guide = example
attention = model(input_ids, age_ids=age_ids, gender_ids=gender_ids, seg_ids=segment_ids, posi_ids=posi_ids, attention_mask=attmask, labels=labels, prior_guide=prior_guide, output_all_encoded_layers=True)[-2]

In [113]:
sentence_b_start = 1
tokens = tokenizer.convert_ids_to_tokens(input_ids.tolist()[0], 'code')
head_view(attention, tokens)

<IPython.core.display.Javascript object>